# 随机化 PCA（Randomized PCA）

## 概述

随机化 PCA 是一种使用随机投影近似主成分的快速算法。当数据维度很高但只需要提取少量主成分时，它比标准 SVD 分解快得多。

## 核心思想

随机化 PCA 的基本思路：

1. 使用随机投影将数据投影到低维子空间
2. 在低维子空间中计算 SVD
3. 将结果映射回原始空间

## 算法优势

- **时间复杂度**：$O(nd^2)$ vs 标准 SVD 的 $O(nd\min(n,d))$
- **内存效率**：只需要存储低秩近似
- **适用场景**：高维数据 + 少量主成分

## 本节内容

1. 随机化 PCA 的使用方法
2. 与标准 PCA 的精度对比
3. 性能（速度）对比
4. MNIST 数据集上的应用

## 1. 环境准备

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml, make_classification
from sklearn.metrics import mean_squared_error

# 设置随机种子
np.random.seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 2. 随机化 PCA 基本使用

在 scikit-learn 中，通过设置 `svd_solver='randomized'` 参数启用随机化 PCA。

In [ ]:
# 生成高维数据用于演示
n_samples = 5000
n_features = 500
n_components = 50

# 生成有内在低维结构的高维数据
X, _ = make_classification(
    n_samples=n_samples, 
    n_features=n_features,
    n_informative=100,
    n_redundant=100,
    n_classes=5,
    random_state=42
)

print(f"数据形状: {X.shape}")
print(f"目标主成分数: {n_components}")

In [ ]:
# 创建随机化 PCA 模型
rnd_pca = PCA(n_components=n_components, svd_solver='randomized', random_state=42)

# 拟合并转换数据
X_reduced = rnd_pca.fit_transform(X)

print(f"降维后数据形状: {X_reduced.shape}")
print(f"保留的方差比: {rnd_pca.explained_variance_ratio_.sum():.4f}")

## 3. 与标准 PCA 的精度对比

比较随机化 PCA 和标准 PCA（完整 SVD）的结果差异。

In [ ]:
# 标准 PCA（完整 SVD）
full_pca = PCA(n_components=n_components, svd_solver='full')
X_full = full_pca.fit_transform(X)

# 比较解释方差
print("解释方差比对比:")
print(f"  随机化 PCA: {rnd_pca.explained_variance_ratio_.sum():.6f}")
print(f"  标准 PCA:   {full_pca.explained_variance_ratio_.sum():.6f}")
print(f"  差异: {abs(rnd_pca.explained_variance_ratio_.sum() - full_pca.explained_variance_ratio_.sum()):.6f}")

# 比较重构误差
X_rnd_reconstructed = rnd_pca.inverse_transform(X_reduced)
X_full_reconstructed = full_pca.inverse_transform(X_full)

mse_rnd = mean_squared_error(X, X_rnd_reconstructed)
mse_full = mean_squared_error(X, X_full_reconstructed)

print(f"\n重构 MSE 对比:")
print(f"  随机化 PCA: {mse_rnd:.6f}")
print(f"  标准 PCA:   {mse_full:.6f}")

In [ ]:
# 可视化解释方差对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 单个成分的方差
ax1 = axes[0]
ax1.plot(range(1, n_components + 1), rnd_pca.explained_variance_ratio_, 
         'o-', label='Randomized PCA', alpha=0.7)
ax1.plot(range(1, n_components + 1), full_pca.explained_variance_ratio_, 
         's--', label='Full PCA', alpha=0.7)
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('Individual Explained Variance')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 累计方差
ax2 = axes[1]
cumsum_rnd = np.cumsum(rnd_pca.explained_variance_ratio_)
cumsum_full = np.cumsum(full_pca.explained_variance_ratio_)
ax2.plot(range(1, n_components + 1), cumsum_rnd, 'o-', label='Randomized PCA', alpha=0.7)
ax2.plot(range(1, n_components + 1), cumsum_full, 's--', label='Full PCA', alpha=0.7)
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Explained Variance')
ax2.set_title('Cumulative Explained Variance')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Randomized PCA vs Full PCA: Accuracy Comparison', fontsize=14)
plt.tight_layout()
plt.show()

## 4. 性能（速度）对比

在不同数据规模下比较随机化 PCA 和标准 PCA 的速度。

In [ ]:
# 测试不同数据规模
feature_sizes = [100, 500, 1000, 2000]
n_samples_test = 3000
n_components_test = 50

results = []

for n_feat in feature_sizes:
    # 生成测试数据
    X_test, _ = make_classification(
        n_samples=n_samples_test, 
        n_features=n_feat,
        n_informative=min(50, n_feat // 2),
        random_state=42
    )
    
    # 测试随机化 PCA
    rnd_pca = PCA(n_components=n_components_test, svd_solver='randomized', random_state=42)
    start = time.time()
    rnd_pca.fit(X_test)
    time_rnd = time.time() - start
    
    # 测试标准 PCA
    full_pca = PCA(n_components=n_components_test, svd_solver='full')
    start = time.time()
    full_pca.fit(X_test)
    time_full = time.time() - start
    
    results.append({
        'features': n_feat,
        'time_randomized': time_rnd,
        'time_full': time_full,
        'speedup': time_full / time_rnd
    })
    
    print(f"特征数: {n_feat:4d} | 随机化: {time_rnd:.3f}s | "
          f"标准: {time_full:.3f}s | 加速比: {time_full/time_rnd:.2f}x")

In [ ]:
# 可视化速度对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

features = [r['features'] for r in results]
times_rnd = [r['time_randomized'] for r in results]
times_full = [r['time_full'] for r in results]
speedups = [r['speedup'] for r in results]

# 运行时间对比
ax1 = axes[0]
x = np.arange(len(features))
width = 0.35
ax1.bar(x - width/2, times_rnd, width, label='Randomized PCA', color='steelblue')
ax1.bar(x + width/2, times_full, width, label='Full PCA', color='coral')
ax1.set_xlabel('Number of Features')
ax1.set_ylabel('Time (seconds)')
ax1.set_title('Execution Time Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(features)
ax1.legend()

# 加速比
ax2 = axes[1]
ax2.bar(x, speedups, color='green', alpha=0.7)
ax2.axhline(y=1, color='red', linestyle='--', label='No speedup')
ax2.set_xlabel('Number of Features')
ax2.set_ylabel('Speedup Factor')
ax2.set_title('Speedup of Randomized PCA over Full PCA')
ax2.set_xticks(x)
ax2.set_xticklabels(features)
ax2.legend()

plt.tight_layout()
plt.show()

## 5. MNIST 数据集应用

在 MNIST 手写数字数据集上应用随机化 PCA。

In [ ]:
# 加载 MNIST 数据集（使用子集）
print("正在加载 MNIST 数据集...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X_mnist = mnist.data[:10000].astype(np.float32) / 255.0
y_mnist = mnist.target[:10000]

print(f"MNIST 数据形状: {X_mnist.shape}")

In [ ]:
# 在 MNIST 上比较随机化 PCA 和标准 PCA
n_components_mnist = 154  # 保留约 95% 方差

# 随机化 PCA
start = time.time()
rnd_pca_mnist = PCA(n_components=n_components_mnist, svd_solver='randomized', random_state=42)
X_rnd_mnist = rnd_pca_mnist.fit_transform(X_mnist)
time_rnd_mnist = time.time() - start

# 标准 PCA
start = time.time()
full_pca_mnist = PCA(n_components=n_components_mnist, svd_solver='full')
X_full_mnist = full_pca_mnist.fit_transform(X_mnist)
time_full_mnist = time.time() - start

print(f"随机化 PCA 时间: {time_rnd_mnist:.3f}s")
print(f"标准 PCA 时间: {time_full_mnist:.3f}s")
print(f"加速比: {time_full_mnist/time_rnd_mnist:.2f}x")
print(f"\n随机化 PCA 保留方差: {rnd_pca_mnist.explained_variance_ratio_.sum():.4f}")
print(f"标准 PCA 保留方差: {full_pca_mnist.explained_variance_ratio_.sum():.4f}")

In [ ]:
# 重构图像对比
X_rnd_recon = rnd_pca_mnist.inverse_transform(X_rnd_mnist)
X_full_recon = full_pca_mnist.inverse_transform(X_full_mnist)

# 选择几张图像展示
n_display = 5
fig, axes = plt.subplots(3, n_display, figsize=(12, 7))

indices = np.random.choice(len(X_mnist), n_display, replace=False)

for i, idx in enumerate(indices):
    # 原始图像
    axes[0, i].imshow(X_mnist[idx].reshape(28, 28), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Original')
    
    # 随机化 PCA 重构
    axes[1, i].imshow(X_rnd_recon[idx].reshape(28, 28), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('Randomized PCA')
    
    # 标准 PCA 重构
    axes[2, i].imshow(X_full_recon[idx].reshape(28, 28), cmap='gray')
    axes[2, i].axis('off')
    if i == 0:
        axes[2, i].set_title('Full PCA')

plt.suptitle(f'MNIST Reconstruction (n_components={n_components_mnist})', fontsize=14)
plt.tight_layout()
plt.show()

# 计算重构误差
mse_rnd = mean_squared_error(X_mnist, X_rnd_recon)
mse_full = mean_squared_error(X_mnist, X_full_recon)
print(f"\n重构 MSE - 随机化 PCA: {mse_rnd:.6f}")
print(f"重构 MSE - 标准 PCA: {mse_full:.6f}")

## 6. SVD 求解器选项

scikit-learn 的 PCA 提供了多种 SVD 求解器：

- **auto**：根据数据自动选择
- **full**：完整 SVD（LAPACK）
- **arpack**：截断 SVD（ARPACK）
- **randomized**：随机化 SVD

In [ ]:
# 比较所有求解器
solvers = ['full', 'arpack', 'randomized']
n_comp = 50

print(f"数据规模: {X_mnist.shape}")
print(f"主成分数: {n_comp}")
print("\n求解器性能对比:")
print("-" * 60)

solver_results = []

for solver in solvers:
    pca = PCA(n_components=n_comp, svd_solver=solver, random_state=42)
    
    start = time.time()
    X_transformed = pca.fit_transform(X_mnist)
    elapsed = time.time() - start
    
    X_recon = pca.inverse_transform(X_transformed)
    mse = mean_squared_error(X_mnist, X_recon)
    
    solver_results.append({
        'solver': solver,
        'time': elapsed,
        'variance': pca.explained_variance_ratio_.sum(),
        'mse': mse
    })
    
    print(f"{solver:12s} | 时间: {elapsed:.3f}s | "
          f"方差: {pca.explained_variance_ratio_.sum():.4f} | MSE: {mse:.6f}")

## 7. 总结

### 关键要点

1. **使用方法**：
   ```python
   PCA(n_components=k, svd_solver='randomized')
   ```

2. **适用场景**：
   - 高维数据（特征数 >> 样本数 或 特征数很大）
   - 只需要提取少量主成分
   - 对计算速度有要求

3. **精度**：
   - 结果是近似的，但通常非常接近精确解
   - 随机性可通过 `random_state` 控制

4. **SVD 求解器选择**：
   - **full**：最精确，适合小数据
   - **arpack**：适合稀疏数据
   - **randomized**：适合高维密集数据
   - **auto**：让 scikit-learn 自动选择

### 注意事项

- 当 `n_components` 接近 `min(n_samples, n_features)` 时，优势不明显
- 结果有随机性，设置 `random_state` 确保可重复
- 对于特别稀疏的数据，考虑使用 `TruncatedSVD`